In [23]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


[notice] A new release of pip is available: 24.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [24]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks and Phishing Scenario Data

Also the data needed to initialise the company working on the phishing scenario

In [25]:
import json
from Utilities.local_make_general import invoke_create_company_context
from Utilities.local_make_general import invoke_save_company_context_block
# from Utilities.local_make_general import invoke_save_user_context_block
from Utilities.local_make_general import invoke_save_user_context_block
from Utilities.local_make_general import invoke_save_team_context_block
from Utilities.util import emulate_ports, unwind_ports, conv
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block
from Utilities.local_make_general import invoke_update_company_relations_block

path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_attributed.json"
identities_data = [
    {
        "who": "me", "email": "SCO/EmailAddr/email_addr_TR_user.json",
        "acct": "SCO/UserAccount/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "step0/TR_Identity"
    },
    {
        "who": "team1", "email": "SCO/EmailAddr/email_addr_team1.json",
        "acct": "SCO/UserAccount/usr_account_team1.json",
        "ident": "SDO/Identity/identity_team1.json",
        "results": "step0/Team1"
    },
    {
        "who": "team2", "email": "SCO/EmailAddr/email_addr_team2.json",
        "acct": "SCO/UserAccount/usr_account_team2.json",
        "ident": "SDO/Identity/identity_team2.json",
        "results": "step0/Team2"
    },
	{
        "who": "team3", "email": "SCO/EmailAddr/email_addr_team3.json",
        "acct": "SCO/UserAccount/usr_account_team3.json",
        "ident": "SDO/Identity/identity_team3.json",
        "results": "step0/Team3"
    },
    {
        "who": "team4", "email": "SCO/EmailAddr/email_addr_team4.json",
        "acct": "SCO/UserAccount/usr_account_team4.json",
        "ident": "SDO/Identity/identity_team4.json",
        "results": "step0/Team4"
    },
    {
        "who": "usr1", "email": "SCO/EmailAddr/email_addr_IT_user1.json",
        "acct": "SCO/UserAccount/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "step0/Usr1"
    },
    {
        "who": "usr2", "email": "SCO/EmailAddr/email_addr_IT_user2.json",
        "acct": "SCO/UserAccount/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "step0/Usr2"
    },
    {
        "who": "usr3", "email": "SCO/EmailAddr/email_addr_IT_user3.json",
        "acct": "SCO/UserAccount/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "step0/Usr3"
    },
    {
        "who": "usr4", "email": "SCO/EmailAddr/email_addr_IT_user4.json",
        "acct": "SCO/UserAccount/usr_account_IT_user4.json",
        "ident": "SDO/Identity/identity_IT_user4.json",
        "results": "step0/Usr4"
    },
    {
        "who": "usr5", "email": "SCO/EmailAddr/email_addr_IT_user5.json",
        "acct": "SCO/UserAccount/usr_account_IT_user5.json",
        "ident": "SDO/Identity/identity_IT_user5.json",
        "results": "step0/Usr5"
    }
]
UserAccount4 = UserAccount(account_type="unix", account_login="sguy", display_name="strange guy")
email_addr4 = EmailAddress(value="strange@mycompany.com", belongs_to_ref=UserAccount4.id)
UserAccount5 = UserAccount(account_type="unix", account_login="dguy", display_name="dumbo guy")
email_addr5 = EmailAddress(value="dumbo@mycompany.com", belongs_to_ref=UserAccount5.id)
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "step0/employer"
}
systems_base = [
    {
        "data_path": "SDO/Identity/identity_Exchange.json",
        "results": "step0/Systems_Exchange"
    }
]
assets_base = [
    {
        "data_path": "SDO/Identity/identity_Laptop1.json",
        "results": "step0/Systems_Laptop1"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop2.json",
        "results": "step0/Systems_Laptop2"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop3.json",
        "results": "step0/Systems_Laptop3"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop4.json",
        "results": "step0/Systems_Laptop4"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop5.json",
        "results": "step0/Systems_Laptop5"
    }
]
results_paths = {}

## B. Create the Multi-Faceted User of this Type Refinery

only perform once on initialisation, otherwise you must update the identity object

### B.1 First Select the User Data

In [26]:
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]


### B.2 Make the User Account for the TR User
Simply pass the paths for the input file and where the results should be written

In [ ]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_user_acct = invoke_make_user_account_block(acct_path, results_path)
# 2. add the record to the in-session bundles and context lists
acct_results_obj_path = results_base + results_path + "__usr_acct.json"
acct_results_context_path = results_base + "step0/context/TR__usr_acct_context.json"
result = invoke_save_user_context_block(acct_results_obj_path, acct_results_context_path)
print(f" result->{result}")

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3', 'user_id': '79563902', 'account_login': 'tjones', 'account_type': 'soc,', 'display_name': 'Trusty Jones'}}


AttributeError: 'NoneType' object has no attribute '__dict__'

### B.3 Make the Email Address for the TR User

In [ ]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_email_addr = invoke_make_email_addr_block(email_path, results_path, TR_user_acct)
# 2. add the record to the in-session bundles and context lists
email_results_obj_path = results_base + results_path + "__email.json"
email_results_context_path = results_base + "step0/context/TR__email_addr_context.json"
result = invoke_save_user_context_block(email_results_obj_path, email_results_context_path)
print(f" result->{result}")

### B.4 Make the Identity for the TR User

In [ ]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_ident = invoke_make_identity_block(ident_path, results_path, email_results=TR_email_addr, acct_results=TR_user_acct)
# 2. add the record to the in-session bundles and context lists
ident_results_obj_path = results_base + results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/TR__ident_context.json"
result = invoke_save_user_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")

## B.5 Mke the Team for the TR User
Simply pass the paths for the input file and where the results should be written

In [ ]:
for ident in identities_data:
    results_paths = {}
    if ident["who"][:3] == "tea":
        # 1. Object Paths and Results Path
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        # 3, Context Pathways
        acct_results_obj_path = results_base + results_path + "__usr_acct.json"
        acct_results_context_path = results_base  + "step0/context/usr_" + results_path[-1:] + "t__usr_acct_context.json"
        email_results_obj_path = results_base + results_path + "__email.json"
        email_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "t__email.json"
        ident_results_obj_path = results_base + results_path + "__ident.json"
        ident_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "t__ident_context.json"
        # 4. invoke the account object using method from ./Utilities/local_make_sco.py, print it and add it to context
        user_acct = invoke_make_user_account_block(acct_path, results_path)
        result = invoke_save_team_context_block(acct_results_obj_path, acct_results_context_path)
        print(f" result->{result}")
        user_email_addr = invoke_make_email_addr_block(email_path, results_path, user_acct)
        result = invoke_save_team_context_block(email_results_obj_path, email_results_context_path)
        print(f" result->{result}")
        user_ident = invoke_make_identity_block(ident_path, results_path,email_results=user_email_addr, acct_results=user_acct)
        result = invoke_save_team_context_block(ident_results_obj_path, ident_results_context_path)
        print(f" result->{result}")

## C. Create the Company

create the company object

In [ ]:
cwd = os.getcwd()
#
# Setup company info into paths
company_ident_path = company_base["ident"]
company_results_path = company_base["results"]
#
# MAke the Company object nd add it to the bundle_list
#
comp_ident = invoke_make_identity_block(company_ident_path, company_results_path)
# 2. add the record to the in-session bundles and context lists
ident_results_obj_path = results_base + company_results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/company__ident_context.json"
result = invoke_create_company_context(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")

## D. Create the Company Users and Add them to Context

For each data record in the list, make the user account, the email-addr and identity

In [ ]:
for ident in identities_data:
    results_paths = {}
    if ident["who"][:3] == "usr":
        # 1. Object Paths and Results Path
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        # 2. Options Context Type
        # 3, Context Pathways
        acct_results_obj_path = results_base + results_path + "__usr_acct.json"
        acct_results_context_path = results_base  + "step0/context/usr_" + results_path[-1:] + "__usr_acct_context.json"
        email_results_obj_path = results_base + results_path + "__email.json"
        email_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__email.json"
        ident_results_obj_path = results_base + results_path + "__ident.json"
        ident_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__ident_context.json"
        # 4. invoke the account object using method from ./Utilities/local_make_sco.py, print it and add it to context
        user_acct = invoke_make_user_account_block(acct_path, results_path)
        result = invoke_save_company_context_block(acct_results_obj_path, acct_results_context_path)
        print(f" result->{result}")
        user_email_addr = invoke_make_email_addr_block(email_path, results_path, user_acct)
        result = invoke_save_company_context_block(email_results_obj_path, email_results_context_path)
        print(f" result->{result}")
        user_ident = invoke_make_identity_block(ident_path, results_path,email_results=user_email_addr, acct_results=user_acct)
        result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path)
        print(f" result->{result}")

## E. Create and Save the List of IT System Identities


In [ ]:
for IT_system in systems_base:
    #
    # Setup company info into paths
    system_ident_path = IT_system["data_path"]
    system_results_path = IT_system["results"]
    # 2. Options Context Type
    #
    # MAke the Company object nd add it to the bundle_list
    #
    system_ident = invoke_make_identity_block(system_ident_path, system_results_path)
    # 2. add the record to the in-session bundles and context lists
    ident_results_obj_path = results_base + system_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/system" + system_results_path[-1:]  + "__ident_context.json"
    result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path)
    print(f" result->{result}")

## F. Create and Save the List of IT Hardware Assets


In [ ]:
for i, asset in enumerate(assets_base):
    #
    # Setup company info into paths
    asset_ident_path = asset["data_path"]
    asset_results_path = asset["results"]

    # 2. Options Context Type
    #
    # MAke the Company object nd add it to the bundle_list
    #
    asset_ident_list = invoke_make_identity_block(asset_ident_path, asset_results_path)
    # 2. add the record to the in-session bundles and context lists
    ident_results_obj_path = results_base + asset_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/hardware" + asset_results_path[-1:]  + str(i) + "__ident_context.json"
    result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path)
    print(f" result->{result}")

## G. Create and Save any SRO's needed


In [ ]:
#
# # Add all relations.contracted-by, employ, asset-of, and system-of
# #
stix_object_path = "../Block_Families/Context/Update_context/relationship_type.json"
results_path = results_base + "step0/_relations_made.json"
# result_string = invoke_update_company_relations_block(stix_object_path, results_path)



# print(f" result->{result}")